In [2]:
# 패키지 로드
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from collections import defaultdict
import os, random

from scipy import sparse
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
from torch.nn.init import normal_
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

from datetime import datetime
from datetime import timedelta

In [3]:
# 경로 설정
data_path = '../data'
saved_path = './saved'
output_path = './submission'

In [4]:
#bd = pd.read_csv('buy_data.csv')
hd = pd.read_csv('history_data.csv')
#mdp = pd.read_csv('meta_data_plus.csv')
#md = pd.read_csv('meta_data.csv')
#prd = pd.read_csv('profile_data.csv')
#sd = pd.read_csv('search_data.csv')
wd = pd.read_csv('watch_e_data.csv')
#ss = pd.read_csv('sample_submission.csv')

merged = pd.read_csv('dfv0.csv')

In [4]:
# 데이터 불러오기 
history = pd.read_csv(os.path.join(data_path, 'history_data.csv'), encoding='utf-8')

search = pd.read_csv(os.path.join(data_path, 'search_data.csv'), encoding='utf-8')
profile = pd.read_csv(os.path.join(data_path, 'profile_data.csv'), encoding='utf-8')
meta = pd.read_csv(os.path.join(data_path, 'meta_data.csv'), encoding='utf-8')
metaplus = pd.read_csv(os.path.join(data_path, 'meta_data_plus.csv'), encoding='utf-8')

In [23]:
datetime_format = "%Y%m%d%H%M%S"

def logtodateint(log) :
    datetime_int = int(log)
    datetime_str = str(datetime_int)
    datetime_result = datetime.strptime(datetime_str[:-2], datetime_format)
    #print(datetime_result)
    return datetime_result

In [5]:
wd = pd.read_csv('wd_v1.csv')
hd = pd.read_csv('hd_v1.csv')

In [6]:
watch = pd.read_csv(os.path.join(data_path, 'wd_v1.csv'), encoding='utf-8')
history = pd.read_csv(os.path.join(data_path, 'hd_v1.csv'), encoding='utf-8') 

In [7]:
watch.head()

,Unnamed: 0,profile_id,act_target_dtl,album_id,watch_time,total_time,continuous_play
0,0,3,49,15,3.850148,3.850148,1
1,1,3,49,16,4.653960,4.663439,1
2,2,3,49,17,4.343805,4.343805,1
3,3,3,49,18,4.219508,4.234107,1
4,4,3,49,19,4.510860,4.510860,0


In [8]:
history.head()

,Unnamed: 0,profile_id,log_time,act_target_dtl,album_id,continuous_play,short_trailer,payment
0,0,3,20220301115719,3,15,1,0,0.0
1,1,3,20220301115809,3,16,1,0,0.0
2,2,3,20220301115958,3,17,1,0,0.0
3,3,3,20220301120118,3,18,1,0,0.0
4,4,3,20220301120229,3,19,1,0,0.0


In [9]:
print(watch.shape)
print(history.shape)

(615480, 7)
(825689, 8)


In [10]:
watch['profile_id']

0             3
1             3
2             3
3             3
4             3
          ...  
615475    33032
615476    33032
615477    33032
615478    33032
615479    33032
Name: profile_id, Length: 615480, dtype: int64

In [11]:
wd_pid = watch['profile_id'].value_counts(dropna=False).sort_index()
wd_pid

3         19
5        436
7          2
16         3
19         2
        ... 
33019      2
33023     10
33026      1
33027      5
33032    144
Name: profile_id, Length: 7658, dtype: int64

In [76]:
#### 정제후 데이터로한거 유실데이터 20454개 

#cnt = 0
#
#for i in wd_pid.index : 
#    tmp_wd = watch[watch['profile_id'] == i]
#    tmp_hd = history[history['profile_id'] == i]
#    
#    
#    #album으로한번 더 포문 돌려서 나누기 x2
 #   for j in tmp_wd.album_id : 
#        
#        tmp_wd_album = tmp_wd[tmp_wd['album_id'] == j]
#        tmp_hd_album = tmp_hd[tmp_hd['album_id'] == j]
#        
#        if tmp_hd_album.index.shape[0] == 0 :
#            cnt += 1 
#            print(cnt ," : album_id ", j ," 는 HD profile_id ", i ,"에 존재하지 않음")
 #           is_album_hd = False
            

In [12]:
history[history['profile_id']==33720]

,Unnamed: 0,profile_id,log_time,act_target_dtl,album_id,continuous_play,short_trailer,payment


## merge한 데이터

In [8]:
merged1 = pd.read_csv(os.path.join(data_path, 'all_df_v2.csv'), encoding='utf-8')
dfv1= pd.read_csv('all_df_v2.csv')

In [155]:
dfv1.head(50)

,Unnamed: 0,profile_id,album_id,ss_id,log_time_x,log_time_y,diff,diff_ss,diff_ox,watch_time,...,search_ox,prefer,sex,age,pr_interest_keyword_cd_1,pr_interest_keyword_cd_2,pr_interest_keyword_cd_3,ch_interest_keyword_cd_1,ch_interest_keyword_cd_2,ch_interest_keyword_cd_3
0,0,3,15.0,2022-03-01 11:56:53,2022-03-01 11:57:19,2022-03-01 11:58:05,0 days 00:00:46,46.0,True,46.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
1,1,3,16.0,2022-03-01 11:56:53,2022-03-01 11:58:09,2022-03-01 11:59:54,0 days 00:01:45,105.0,False,104.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
2,2,3,17.0,2022-03-01 11:56:53,2022-03-01 11:59:58,2022-03-01 12:01:14,0 days 00:01:16,76.0,True,76.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
3,3,3,18.0,2022-03-01 11:56:53,2022-03-01 12:01:18,2022-03-01 12:02:26,0 days 00:01:08,68.0,False,67.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
4,4,3,19.0,2022-03-01 11:56:53,2022-03-01 12:02:29,2022-03-01 12:04:00,0 days 00:01:31,91.0,False,90.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
5,5,3,20.0,2022-03-01 11:56:53,2022-03-01 12:04:50,2022-03-01 12:06:50,0 days 00:02:00,120.0,False,119.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
6,6,3,20.0,2022-03-01 11:56:53,2022-03-01 12:04:50,2022-03-01 12:34:08,0 days 00:29:18,1758.0,False,119.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
7,7,3,20.0,2022-03-01 11:56:53,2022-03-01 12:31:56,2022-03-01 12:06:50,-1 days +23:34:54,-1506.0,False,119.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
8,8,3,20.0,2022-03-01 11:56:53,2022-03-01 12:31:56,2022-03-01 12:34:08,0 days 00:02:12,132.0,False,119.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
9,9,3,21.0,2022-03-01 11:56:53,2022-03-01 12:06:54,2022-03-01 12:08:54,0 days 00:02:00,120.0,False,119.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04


In [50]:
dfv2 = dfv1.copy()

In [145]:
dfv2['act_target_dtl'].value_countsunts()

MKID049    1954869
Name: act_target_dtl, dtype: int64

In [10]:
dfv1.isnull().sum().to_frame()

,0
Unnamed: 0,0
profile_id,0
album_id,26
ss_id,26
log_time_x,39901
log_time_y,101969
diff,141844
diff_ss,141844
diff_ox,0
watch_time,101969


In [30]:
dfv1[['profile_id','album_id','ss_id','log_time_x','log_time_y','diff_ss','diff_ox','act_target_dtl_x','act_target_dtl','watch_time']].head(10)

,profile_id,album_id,ss_id,log_time_x,log_time_y,diff_ss,diff_ox,act_target_dtl_x,act_target_dtl,watch_time
0,3,15.0,2022-03-01 11:56:53,2022-03-01 11:57:19,2022-03-01 11:58:05,46.0,True,MKID003,MKID049,46.0
1,3,16.0,2022-03-01 11:56:53,2022-03-01 11:58:09,2022-03-01 11:59:54,105.0,False,MKID003,MKID049,104.0
2,3,17.0,2022-03-01 11:56:53,2022-03-01 11:59:58,2022-03-01 12:01:14,76.0,True,MKID003,MKID049,76.0
3,3,18.0,2022-03-01 11:56:53,2022-03-01 12:01:18,2022-03-01 12:02:26,68.0,False,MKID003,MKID049,67.0
4,3,19.0,2022-03-01 11:56:53,2022-03-01 12:02:29,2022-03-01 12:04:00,91.0,False,MKID003,MKID049,90.0
5,3,20.0,2022-03-01 11:56:53,2022-03-01 12:04:50,2022-03-01 12:06:50,120.0,False,MKID003,MKID049,119.0
6,3,20.0,2022-03-01 11:56:53,2022-03-01 12:04:50,2022-03-01 12:34:08,1758.0,False,MKID003,MKID049,119.0
7,3,20.0,2022-03-01 11:56:53,2022-03-01 12:31:56,2022-03-01 12:06:50,-1506.0,False,MKID003,MKID049,119.0
8,3,20.0,2022-03-01 11:56:53,2022-03-01 12:31:56,2022-03-01 12:34:08,132.0,False,MKID003,MKID049,119.0
9,3,21.0,2022-03-01 11:56:53,2022-03-01 12:06:54,2022-03-01 12:08:54,120.0,False,MKID003,MKID049,119.0


In [20]:
#act_target_dtl_x 

dfv1.act_target_dtl_x = 'MKID003'

In [31]:
temp = dfv1[['profile_id','album_id','ss_id','log_time_x','log_time_y','diff_ss','diff_ox','act_target_dtl_x','act_target_dtl','watch_time']]

In [51]:
temp[temp['log_time_x'].isnull()]

,profile_id,album_id,ss_id,log_time_x,log_time_y,diff_ss,diff_ox,act_target_dtl_x,act_target_dtl,watch_time
5044,24,3692.0,2022-04-28 00:34:49,NaN,2022-04-28 00:35:24,NaN,False,MKID003,MKID049,11.0
5138,30,1902.0,2022-03-05 13:00:48,NaN,2022-03-05 13:29:36,NaN,False,MKID003,MKID049,659.0
5139,30,1903.0,2022-03-05 13:00:48,NaN,2022-03-05 13:40:34,NaN,False,MKID003,MKID049,659.0
5140,30,1908.0,2022-03-20 14:00:09,NaN,2022-03-20 14:47:07,NaN,False,MKID003,MKID049,664.0
5141,30,1909.0,2022-03-28 12:12:30,NaN,2022-03-28 12:13:08,NaN,False,MKID003,MKID049,53.0
...,...,...,...,...,...,...,...,...,...,...
2056833,33032,373.0,2022-04-27 15:50:09,NaN,2022-04-27 15:51:12,NaN,False,MKID003,MKID049,7.0
2056834,33032,3128.0,2022-04-27 15:50:09,NaN,2022-04-27 15:51:53,NaN,False,MKID003,MKID049,2.0
2056835,33032,393.0,2022-04-27 15:50:09,NaN,2022-04-27 15:53:08,NaN,False,MKID003,MKID049,81.0
2056836,33032,3625.0,2022-04-27 15:50:09,NaN,2022-04-27 15:54:57,NaN,False,MKID003,MKID049,9.0


In [65]:
tmpindex = temp[temp['log_time_x'].isnull() & temp['log_time_y'].notnull() & temp['watch_time'].notnull()].index
tmpindex

Int64Index([  69619,   69620,   69621,   69622,   69623,   69624,   69625,
              69626,   69627,   69628,
            ...
            2056828, 2056829, 2056830, 2056831, 2056832, 2056833, 2056834,
            2056835, 2056836, 2056837],
           dtype='int64', length=38921)

In [66]:
%%time

## log_time_x 빈거/ log_time_y, watch_time 안빈거 -> log_time_x채우기  38921개

for i in tmpindex : 
    print(i)
    temp.loc[i,'log_time_x'] = datetime.strptime(temp.loc[i,'log_time_y'], '%Y-%m-%d %H:%M:%S') - timedelta(seconds= int(temp.loc[i].watch_time))
    temp.loc[i,'log_time_x'] = temp.loc[i,'log_time_x'].strftime('%Y-%m-%d %H:%M:%S')
    
temp

69619
69620
69621
69622
69623
69624
69625
69626
69627
69628
69629
69630
69631
69632
69633
69634
69635
69636
70250
70251
70252
70253
70254
70255
70256
70257
70258
70259
70260
70261
70262
70263
70264
70265
70266
70267
70268
70269
70270
70271
70272
70860
70861
70862
70863
70864
70865
72459
72460
72461
72462
72463
72464
72465
72466
72467
72468
72469
72470
72471
72472
72473
72474
72475
72476
72477
72478
72479
72480
72481
72482
72483
72484
72485
72486
72487
72488
72489
72490
72491
72492
72493
72494
72495
72496
72497
72498
72499
72500
72501
72502
72503
72504
72505
72506
72507
72508
72509
72510
72511
72512
72513
72514
72515
72516
72517
72518
72519
72520
72521
72522
72523
72524
72525
72526
72527
72528
72529
72530
72531
72532
72533
72534
72535
72536
72537
72538
72539
72540
72541
72542
72543
72544
72545
72546
72547
72548
72549
72550
72551
72552
72553
72554
72555
72556
72557
72558
72559
72560
72561
72562
72563
72564
72565
72566
72567
72568
72569
72570
72571
72572
72573
72574
72575
72576
72577
7257

119964
119965
119966
119967
119968
119969
119970
119971
119972
119973
119974
119975
119976
119977
119978
119979
119980
119981
120547
121228
121256
121257
121258
121259
121260
121261
121262
122004
122005
122006
122007
122008
122009
122010
122011
122012
122013
122014
122015
122016
122017
122018
122019
122020
122021
122022
122023
122024
122025
122026
122027
122028
122029
122030
122031
122032
122033
122034
122035
122036
122037
122038
122039
122040
122041
122042
122043
122044
122045
122046
122047
122048
122049
122050
122051
122052
122053
122054
122055
122056
122057
122058
122059
122060
122061
122062
122063
122064
122065
122066
122067
122081
122082
122083
122344
122345
122346
122347
122348
122349
122350
124857
124858
124859
124860
124861
124862
124863
124864
124865
124866
124867
124868
124869
124870
124871
124872
124873
124874
124875
124876
124877
124878
124879
124880
124881
124882
124883
124884
124885
124886
124887
124888
124889
124890
124891
124892
124893
124894
124895
124896
124897
124898

185980
185981
185982
185983
185984
185985
185986
185987
185988
185989
185990
185991
185992
185993
185994
185995
185996
185997
185998
185999
186000
186001
186002
186003
186004
186005
186006
186007
186008
186009
186010
186011
186012
186013
186014
186015
186016
186017
186018
186019
186020
186021
186022
186023
186024
186025
186026
186027
186028
186029
186030
186031
186032
186033
186034
186035
186036
186037
186038
186039
186040
186041
186042
186043
186044
186045
186046
186047
186048
186049
186050
186051
186052
186053
186054
186055
186056
186057
186058
186059
186060
186061
186062
186063
186064
186065
186066
186067
186068
186069
186070
186071
186072
186073
186074
186075
186076
186077
186078
186079
186080
186081
186082
186083
186084
186085
186086
186087
186088
186089
186090
186091
186092
186093
186094
186095
186096
186097
186098
186099
186100
186101
186102
186103
186104
186105
186106
186107
186108
186109
186110
186111
186112
186113
186114
186115
186116
186117
186118
186119
186120
186121
186122

297294
297295
297296
297297
297298
297299
297300
297301
297302
297303
297304
297305
299412
299413
299414
299415
299416
299417
299418
299419
299420
299421
299422
299423
299424
299425
299426
299427
299529
299530
299531
299532
299533
299534
299535
299536
299537
299538
299539
299540
299541
299542
299543
299544
299545
299546
299547
299548
299549
299550
299551
299552
300749
300750
300751
300752
300753
300754
300755
300756
300757
300758
300759
300760
300761
300762
300763
300764
300765
300766
300767
300768
300769
300770
300771
300772
300773
300774
300775
300776
300777
300778
300779
300780
300781
300782
300783
300784
300785
300786
300787
300788
300789
300790
300791
300792
300793
300794
300795
300796
300797
300798
300799
300800
300801
300802
300803
300804
300805
300806
300807
300808
300809
300810
300811
300812
300813
300814
300815
300816
300817
300818
300819
300820
300821
300822
300823
300824
300825
300826
300827
300828
300829
300830
300831
300832
300833
300834
300835
300836
300837
300838
300839

412728
412729
412730
412731
412732
412733
412734
412735
412736
412737
412738
412739
412740
412741
412742
412743
412744
412745
412746
412747
412748
412749
412750
412751
412752
412753
412754
412755
412756
412757
412758
412759
412760
412761
412762
412763
412764
412765
412766
412767
412768
412769
412770
412771
412772
412773
412774
412775
412776
412777
412778
412779
412780
412781
412782
412783
412784
412785
412786
412787
412788
412789
412790
412791
412792
412793
412794
412795
412796
412797
412798
412799
412800
412801
412802
412803
412804
412805
412806
412807
412808
412809
412810
412811
412812
412813
412814
412815
412816
412817
412818
412819
412820
412821
412822
412823
412824
412825
412826
412827
412828
412829
412830
412831
412832
412833
412834
412835
412836
412837
412838
412839
412840
412841
412842
412843
412844
412845
412846
412847
412848
412849
412850
412851
412852
412853
412854
412855
412856
412857
412858
412859
412860
412861
412862
412863
412864
412865
412866
412867
412868
412869
412870

...

1819528
1819529
1819530
1819531
1830940
1831076
1831623
1831624
1831625
1831626
1831627
1831628
1831629
1831630
1831631
1831632
1831633
1831634
1831635
1831636
1831637
1831638
1831639
1831640
1831641
1831642
1831643
1831644
1831645
1831646
1831647
1831648
1831649
1831650
1831651
1831652
1831653
1831654
1831655
1831656
1831657
1831658
1831659
1831660
1831661
1831662
1831663
1831664
1831665
1831666
1831667
1831668
1831669
1831670
1831671
1831672
1831673
1831674
1831675
1831676
1831677
1831678
1831679
1831680
1831681
1831682
1831683
1831684
1831685
1831686
1831687
1831688
1831689
1831690
1831691
1831692
1831693
1831694
1831695
1831696
1831697
1831698
1831699
1831700
1831701
1831702
1831703
1831704
1831705
1831706
1831707
1831708
1831709
1831710
1831711
1831712
1831713
1831714
1831715
1831716
1831717
1831718
1831719
1831720
1831721
1831722
1831723
1831724
1831725
1831726
1831727
1831728
1831729
1831730
1831731
1831732
1831733
1831734
1831735
1831736
1831737
1831738
1831739
1831740
1831741


1935515
1935516
1935517
1935518
1935519
1935520
1935521
1935522
1935523
1935524
1935525
1935526
1935527
1935528
1935529
1935530
1935531
1935532
1935533
1939788
1939789
1939790
1939791
1939792
1939793
1939794
1939795
1939796
1939891
1939892
1939893
1939894
1939895
1939896
1939897
1939898
1939899
1939900
1941709
1941710
1941711
1941712
1941713
1941714
1941715
1941716
1941717
1941718
1941719
1941720
1941721
1941722
1942204
1942205
1942206
1942207
1942208
1942209
1942210
1942211
1942212
1942213
1942214
1942215
1942216
1942217
1942218
1942219
1942220
1942221
1942495
1942496
1942497
1942919
1942920
1942921
1942922
1942923
1942924
1942925
1942926
1942927
1944081
1944129
1944130
1944131
1944236
1944237
1944238
1944239
1944240
1944241
1944242
1944243
1944244
1944245
1944246
1944247
1944248
1944249
1944250
1944251
1944252
1944253
1944254
1944255
1944256
1944409
1944410
1944411
1944412
1944413
1944414
1944415
1944416
1944417
1944418
1944419
1944420
1944421
1944422
1944423
1944424
1944425
1944426


1980141
1980142
1980143
1980144
1980145
1980146
1980147
1980148
1980149
1980150
1980151
1980152
1980153
1980154
1980155
1980156
1980157
1980158
1980159
1980160
1980161
1980162
1980163
1980164
1980165
1980166
1980167
1980168
1980169
1980170
1980171
1980172
1980173
1980174
1980175
1980176
1980177
1980178
1980179
1980180
1980181
1980182
1980183
1980184
1980185
1980186
1980187
1980188
1980189
1980190
1980191
1980192
1980193
1980194
1980195
1980196
1980197
1980198
1980199
1980200
1980201
1980202
1980203
1980204
1980205
1980206
1980207
1980208
1980209
1980210
1980211
1980212
1980213
1980214
1980215
1980216
1980217
1980218
1980219
1980220
1980221
1980222
1980223
1980224
1980225
1981122
1981123
1981124
1981125
1981126
1981127
1981128
1981129
1981130
1981283
1981284
1981285
1981286
1981287
1981288
1981289
1981621
1982569
1982570
1982571
1982572
1982573
1982574
1982575
1982576
1982577
1982578
1982579
1982580
1982581
1982582
1982583
1982584
1982585
1982586
1982587
1982588
1983431
1983432
1983433


2035415
2035416
2035417
2035418
2035419
2035420
2035421
2035422
2035423
2035424
2035425
2035426
2035427
2035428
2035429
2035430
2035431
2035432
2035433
2035434
2035435
2035436
2035437
2035438
2035439
2035440
2035441
2035442
2035443
2035444
2035445
2035446
2035447
2035448
2035449
2035450
2035451
2035452
2035453
2035454
2035455
2035456
2035457
2035458
2035459
2035460
2035461
2035462
2035463
2035464
2035465
2035466
2035467
2035468
2035469
2035470
2035471
2035472
2035473
2035474
2035475
2035476
2035477
2035478
2035479
2035480
2035481
2035482
2035483
2035484
2035485
2035486
2035487
2035488
2035489
2035490
2035491
2035492
2035493
2035494
2035495
2035496
2035497
2035498
2035499
2035500
2035501
2035502
2035503
2035504
2035505
2035506
2035507
2035508
2035509
2035510
2035511
2035512
2035513
2035514
2035515
2035516
2035517
2035518
2035519
2035520
2035521
2035522
2035523
2035524
2035525
2035526
2035527
2035528
2035529
2035530
2035531
2035532
2035533
2035534
2035535
2035536
2035537
2035538
2035539


,profile_id,album_id,ss_id,log_time_x,log_time_y,diff_ss,diff_ox,act_target_dtl_x,act_target_dtl,watch_time
0,3,15.0,2022-03-01 11:56:53,2022-03-01 11:57:19,2022-03-01 11:58:05,46.0,True,MKID003,MKID049,46.0
1,3,16.0,2022-03-01 11:56:53,2022-03-01 11:58:09,2022-03-01 11:59:54,105.0,False,MKID003,MKID049,104.0
2,3,17.0,2022-03-01 11:56:53,2022-03-01 11:59:58,2022-03-01 12:01:14,76.0,True,MKID003,MKID049,76.0
3,3,18.0,2022-03-01 11:56:53,2022-03-01 12:01:18,2022-03-01 12:02:26,68.0,False,MKID003,MKID049,67.0
4,3,19.0,2022-03-01 11:56:53,2022-03-01 12:02:29,2022-03-01 12:04:00,91.0,False,MKID003,MKID049,90.0
...,...,...,...,...,...,...,...,...,...,...
2056833,33032,373.0,2022-04-27 15:50:09,2022-04-27 15:51:05,2022-04-27 15:51:12,NaN,False,MKID003,MKID049,7.0
2056834,33032,3128.0,2022-04-27 15:50:09,2022-04-27 15:51:51,2022-04-27 15:51:53,NaN,False,MKID003,MKID049,2.0
2056835,33032,393.0,2022-04-27 15:50:09,2022-04-27 15:51:47,2022-04-27 15:53:08,NaN,False,MKID003,MKID049,81.0
2056836,33032,3625.0,2022-04-27 15:50:09,2022-04-27 15:54:48,2022-04-27 15:54:57,NaN,False,MKID003,MKID049,9.0


In [73]:
## log_time_y 빈거/ log_time_x, watch_time 안빈거 -> log_time_y채우기  -> 없음 

temp[temp['log_time_y'].isnull() & temp['log_time_x'].notnull() & temp['watch_time'].notnull()]


,profile_id,album_id,ss_id,log_time_x,log_time_y,diff_ss,diff_ox,act_target_dtl_x,act_target_dtl,watch_time


In [71]:
## watch_time 빈거/ log_time_x, log_time_y 안빈거 -> watch_time채우기  -> 없음
##원래 이걸로 채우면안되는데 어짜피 채울게 없었음..

temp[temp['watch_time'].isnull() & temp['log_time_x'].notnull() & temp['log_time_y'].notnull()]

,profile_id,album_id,ss_id,log_time_x,log_time_y,diff_ss,diff_ox,act_target_dtl_x,act_target_dtl,watch_time


In [74]:
## watch_time이랑 log_time_y 가 둘다 비어서 결측치 못채우는 케이스 101943개
## log_time_x 만 빈거 26개랑 더하면 101969개 나옴

temp[temp['watch_time'].isnull() & temp['log_time_x'].notnull() & temp['log_time_y'].isnull()]

,profile_id,album_id,ss_id,log_time_x,log_time_y,diff_ss,diff_ox,act_target_dtl_x,act_target_dtl,watch_time
24,3,33.0,2022-03-01 11:56:53,2022-03-01 12:35:05,NaN,NaN,False,MKID003,NaN,NaN
35,5,73.0,2022-03-02 21:51:29,2022-03-04 18:39:17,NaN,NaN,False,MKID003,NaN,NaN
116,5,129.0,2022-03-08 08:44:09,2022-03-09 09:04:09,NaN,NaN,False,MKID003,NaN,NaN
343,5,180.0,2022-03-21 19:37:33,2022-03-21 20:03:19,NaN,NaN,False,MKID003,NaN,NaN
406,5,196.0,2022-03-22 09:15:58,2022-03-22 18:15:00,NaN,NaN,False,MKID003,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2056686,33032,14662.0,2022-04-27 11:21:16,2022-04-27 11:23:35,NaN,NaN,False,MKID003,NaN,NaN
2056687,33032,673.0,2022-04-27 11:21:16,2022-04-27 11:24:09,NaN,NaN,False,MKID003,NaN,NaN
2056692,33032,320.0,2022-04-27 11:21:16,2022-04-27 11:28:37,NaN,NaN,False,MKID003,NaN,NaN
2056718,33032,65.0,2022-04-27 14:56:06,2022-04-27 14:56:34,NaN,NaN,False,MKID003,NaN,NaN


In [78]:
# diff_ss, log_time_y 둘다 빈거 101969개
temp[temp['diff_ss'].isnull() & temp['log_time_y'].isnull()]

,profile_id,album_id,ss_id,log_time_x,log_time_y,diff_ss,diff_ox,act_target_dtl_x,act_target_dtl,watch_time
24,3,33.0,2022-03-01 11:56:53,2022-03-01 12:35:05,NaN,NaN,False,MKID003,NaN,NaN
35,5,73.0,2022-03-02 21:51:29,2022-03-04 18:39:17,NaN,NaN,False,MKID003,NaN,NaN
116,5,129.0,2022-03-08 08:44:09,2022-03-09 09:04:09,NaN,NaN,False,MKID003,NaN,NaN
343,5,180.0,2022-03-21 19:37:33,2022-03-21 20:03:19,NaN,NaN,False,MKID003,NaN,NaN
406,5,196.0,2022-03-22 09:15:58,2022-03-22 18:15:00,NaN,NaN,False,MKID003,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2056686,33032,14662.0,2022-04-27 11:21:16,2022-04-27 11:23:35,NaN,NaN,False,MKID003,NaN,NaN
2056687,33032,673.0,2022-04-27 11:21:16,2022-04-27 11:24:09,NaN,NaN,False,MKID003,NaN,NaN
2056692,33032,320.0,2022-04-27 11:21:16,2022-04-27 11:28:37,NaN,NaN,False,MKID003,NaN,NaN
2056718,33032,65.0,2022-04-27 14:56:06,2022-04-27 14:56:34,NaN,NaN,False,MKID003,NaN,NaN


In [83]:
temp.isnull().sum().to_frame()

,0
profile_id,0
album_id,26
ss_id,26
log_time_x,26
log_time_y,101969
diff_ss,141844
diff_ox,0
act_target_dtl_x,0
act_target_dtl,101969
watch_time,101969


In [82]:
# diff_ss, log_time_x 둘다 빈거 26개
print(temp[temp['diff_ss'].isnull() & temp['log_time_x'].isnull()].shape)
temp[temp['diff_ss'].isnull() & temp['log_time_x'].isnull()]


(26, 10)


,profile_id,album_id,ss_id,log_time_x,log_time_y,diff_ss,diff_ox,act_target_dtl_x,act_target_dtl,watch_time
69493,931,NaN,NaN,NaN,NaN,NaN,False,MKID003,NaN,NaN
103292,1713,NaN,NaN,NaN,NaN,NaN,False,MKID003,NaN,NaN
550471,5650,NaN,NaN,NaN,NaN,NaN,False,MKID003,NaN,NaN
586548,6097,NaN,NaN,NaN,NaN,NaN,False,MKID003,NaN,NaN
620188,6132,NaN,NaN,NaN,NaN,NaN,False,MKID003,NaN,NaN
709992,7229,NaN,NaN,NaN,NaN,NaN,False,MKID003,NaN,NaN
710256,7250,NaN,NaN,NaN,NaN,NaN,False,MKID003,NaN,NaN
727102,7409,NaN,NaN,NaN,NaN,NaN,False,MKID003,NaN,NaN
794712,8199,NaN,NaN,NaN,NaN,NaN,False,MKID003,NaN,NaN
895791,9552,NaN,NaN,NaN,NaN,NaN,False,MKID003,NaN,NaN


In [90]:
## diff_ss 빈거 / log_time_x log_time_y 안 빈거 39875개

tmpindex1 = temp[temp['diff_ss'].isnull() & temp['log_time_x'].notnull() & temp['log_time_y'].notnull() ].index
tmpindex1

Int64Index([   5044,    5138,    5139,    5140,    5141,    5142,    5143,
               5144,    5145,    5146,
            ...
            2056828, 2056829, 2056830, 2056831, 2056832, 2056833, 2056834,
            2056835, 2056836, 2056837],
           dtype='int64', length=39875)

In [119]:
# diff 빼먹어서 추가 

temp['diff'] = dfv1['diff']
temp

,profile_id,album_id,ss_id,log_time_x,log_time_y,diff_ss,diff_ox,act_target_dtl_x,act_target_dtl,watch_time,diff
0,3,15.0,2022-03-01 11:56:53,2022-03-01 11:57:19,2022-03-01 11:58:05,46.0,True,MKID003,MKID049,46.0,0 days 00:00:46
1,3,16.0,2022-03-01 11:56:53,2022-03-01 11:58:09,2022-03-01 11:59:54,105.0,False,MKID003,MKID049,104.0,0 days 00:01:45
2,3,17.0,2022-03-01 11:56:53,2022-03-01 11:59:58,2022-03-01 12:01:14,76.0,True,MKID003,MKID049,76.0,0 days 00:01:16
3,3,18.0,2022-03-01 11:56:53,2022-03-01 12:01:18,2022-03-01 12:02:26,68.0,False,MKID003,MKID049,67.0,0 days 00:01:08
4,3,19.0,2022-03-01 11:56:53,2022-03-01 12:02:29,2022-03-01 12:04:00,91.0,False,MKID003,MKID049,90.0,0 days 00:01:31
...,...,...,...,...,...,...,...,...,...,...,...
2056833,33032,373.0,2022-04-27 15:50:09,2022-04-27 15:51:05,2022-04-27 15:51:12,0:00:07,False,MKID003,MKID049,7.0,NaN
2056834,33032,3128.0,2022-04-27 15:50:09,2022-04-27 15:51:51,2022-04-27 15:51:53,0:00:02,False,MKID003,MKID049,2.0,NaN
2056835,33032,393.0,2022-04-27 15:50:09,2022-04-27 15:51:47,2022-04-27 15:53:08,0:01:21,False,MKID003,MKID049,81.0,NaN
2056836,33032,3625.0,2022-04-27 15:50:09,2022-04-27 15:54:48,2022-04-27 15:54:57,0:00:09,False,MKID003,MKID049,9.0,NaN


In [120]:
%%time
#timedelta는 float로 리턴됨

for i in tmpindex1 : 
    temp.loc[i,'diff'] = datetime.strptime(temp.loc[i,'log_time_y'], '%Y-%m-%d %H:%M:%S') - datetime.strptime(temp.loc[i,'log_time_x'], '%Y-%m-%d %H:%M:%S') 
    temp.loc[i,'diff_ss'] = temp.loc[i,'diff'].total_seconds()

CPU times: user 5min 12s, sys: 1.04 s, total: 5min 13s
Wall time: 5min 13s


In [121]:
temp.loc[tmpindex1]

,profile_id,album_id,ss_id,log_time_x,log_time_y,diff_ss,diff_ox,act_target_dtl_x,act_target_dtl,watch_time,diff
5044,24,3692.0,2022-04-28 00:34:49,2022-04-28 00:35:13,2022-04-28 00:35:24,11.0,False,MKID003,MKID049,11.0,0:00:11
5138,30,1902.0,2022-03-05 13:00:48,2022-03-05 13:18:37,2022-03-05 13:29:36,659.0,False,MKID003,MKID049,659.0,0:10:59
5139,30,1903.0,2022-03-05 13:00:48,2022-03-05 13:29:35,2022-03-05 13:40:34,659.0,False,MKID003,MKID049,659.0,0:10:59
5140,30,1908.0,2022-03-20 14:00:09,2022-03-20 14:36:03,2022-03-20 14:47:07,664.0,False,MKID003,MKID049,664.0,0:11:04
5141,30,1909.0,2022-03-28 12:12:30,2022-03-28 12:12:15,2022-03-28 12:13:08,53.0,False,MKID003,MKID049,53.0,0:00:53
...,...,...,...,...,...,...,...,...,...,...,...
2056833,33032,373.0,2022-04-27 15:50:09,2022-04-27 15:51:05,2022-04-27 15:51:12,7.0,False,MKID003,MKID049,7.0,0:00:07
2056834,33032,3128.0,2022-04-27 15:50:09,2022-04-27 15:51:51,2022-04-27 15:51:53,2.0,False,MKID003,MKID049,2.0,0:00:02
2056835,33032,393.0,2022-04-27 15:50:09,2022-04-27 15:51:47,2022-04-27 15:53:08,81.0,False,MKID003,MKID049,81.0,0:01:21
2056836,33032,3625.0,2022-04-27 15:50:09,2022-04-27 15:54:48,2022-04-27 15:54:57,9.0,False,MKID003,MKID049,9.0,0:00:09


In [126]:
temp

,profile_id,album_id,ss_id,log_time_x,log_time_y,diff_ss,diff_ox,act_target_dtl_x,act_target_dtl,watch_time,diff
0,3,15.0,2022-03-01 11:56:53,2022-03-01 11:57:19,2022-03-01 11:58:05,46.0,True,MKID003,MKID049,46.0,0 days 00:00:46
1,3,16.0,2022-03-01 11:56:53,2022-03-01 11:58:09,2022-03-01 11:59:54,105.0,False,MKID003,MKID049,104.0,0 days 00:01:45
2,3,17.0,2022-03-01 11:56:53,2022-03-01 11:59:58,2022-03-01 12:01:14,76.0,True,MKID003,MKID049,76.0,0 days 00:01:16
3,3,18.0,2022-03-01 11:56:53,2022-03-01 12:01:18,2022-03-01 12:02:26,68.0,False,MKID003,MKID049,67.0,0 days 00:01:08
4,3,19.0,2022-03-01 11:56:53,2022-03-01 12:02:29,2022-03-01 12:04:00,91.0,False,MKID003,MKID049,90.0,0 days 00:01:31
...,...,...,...,...,...,...,...,...,...,...,...
2056833,33032,373.0,2022-04-27 15:50:09,2022-04-27 15:51:05,2022-04-27 15:51:12,7.0,False,MKID003,MKID049,7.0,0:00:07
2056834,33032,3128.0,2022-04-27 15:50:09,2022-04-27 15:51:51,2022-04-27 15:51:53,2.0,False,MKID003,MKID049,2.0,0:00:02
2056835,33032,393.0,2022-04-27 15:50:09,2022-04-27 15:51:47,2022-04-27 15:53:08,81.0,False,MKID003,MKID049,81.0,0:01:21
2056836,33032,3625.0,2022-04-27 15:50:09,2022-04-27 15:54:48,2022-04-27 15:54:57,9.0,False,MKID003,MKID049,9.0,0:00:09


In [151]:
type(temp.loc[0].diff)

method

In [122]:
temp.isnull().sum().to_frame()

,0
profile_id,0
album_id,26
ss_id,26
log_time_x,26
log_time_y,101969
diff_ss,101969
diff_ox,0
act_target_dtl_x,0
act_target_dtl,101969
watch_time,101969


In [127]:
%%time 
## diff 비워져있어서 diff_ox가 false로 뜨는게 더 많았을 것으로 예상됨
## diff_ss새로 채워진거만 diff_ox 다시 계산

for i in tmpindex1 : 
    if temp.loc[i, 'diff_ss'] == temp.loc[i, 'watch_time'] :
        temp.loc[i,'diff_ox'] = True
    else : 
        temp.loc[i,'diff_ox'] = False

CPU times: user 6.15 s, sys: 19.7 ms, total: 6.16 s
Wall time: 6.16 s


In [130]:
dfv1['diff_ox'].value_counts()

False    1813905
True      242933
Name: diff_ox, dtype: int64

In [132]:
temp['diff_ox'].value_counts()
## 4만개정도 늘어남

False    1774030
True      282808
Name: diff_ox, dtype: int64

In [104]:
##diff_ox : diff_ss 랑 watch_time이 같으면 True 다르면 False //  > 빈값들이 다 False로 들어가있음

In [ ]:
##작업한거 dfv2에 넣기
# log_time_y - watch_time = log_time_x 
# 식 바꿔서 log_time_y, watch_time 채우려 했으나 두 항이 비어서 못채운거 101969개
# diff, diff_ss, diff_ox 채움

In [123]:
dfv2['log_time_x'] = temp['log_time_x']

In [133]:
dfv2['diff_ss'] = temp['diff_ss']
dfv2['diff'] = temp['diff']
dfv2['diff_ox'] = temp['diff_ox']

In [134]:
dfv2.isnull().sum().to_frame()

,0
Unnamed: 0,0
profile_id,0
album_id,26
ss_id,26
log_time_x,26
log_time_y,101969
diff,101969
diff_ss,101969
diff_ox,0
watch_time,101969


In [146]:
dfv2.act_target_dtl_x = 'MKID003'
dfv2.act_target_dtl = 'MKID049'

In [156]:
dfv3 = dfv2.copy()

In [158]:
dfv3.head()

,Unnamed: 0,profile_id,album_id,ss_id,log_time_x,log_time_y,diff,diff_ss,diff_ox,watch_time,...,search_ox,prefer,sex,age,pr_interest_keyword_cd_1,pr_interest_keyword_cd_2,pr_interest_keyword_cd_3,ch_interest_keyword_cd_1,ch_interest_keyword_cd_2,ch_interest_keyword_cd_3
0,0,3,15.0,2022-03-01 11:56:53,2022-03-01 11:57:19,2022-03-01 11:58:05,0 days 00:00:46,46.0,True,46.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
1,1,3,16.0,2022-03-01 11:56:53,2022-03-01 11:58:09,2022-03-01 11:59:54,0 days 00:01:45,105.0,False,104.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
2,2,3,17.0,2022-03-01 11:56:53,2022-03-01 11:59:58,2022-03-01 12:01:14,0 days 00:01:16,76.0,True,76.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
3,3,3,18.0,2022-03-01 11:56:53,2022-03-01 12:01:18,2022-03-01 12:02:26,0 days 00:01:08,68.0,False,67.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
4,4,3,19.0,2022-03-01 11:56:53,2022-03-01 12:02:29,2022-03-01 12:04:00,0 days 00:01:31,91.0,False,90.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04


In [163]:
sample = dfv3[dfv3['diff_ss'] - dfv3['watch_time'] > 20]
sample = dfv3[dfv3['diff_ss'] - dfv3['watch_time'] < -20]
sample

,Unnamed: 0,profile_id,album_id,ss_id,log_time_x,log_time_y,diff,diff_ss,diff_ox,watch_time,...,search_ox,prefer,sex,age,pr_interest_keyword_cd_1,pr_interest_keyword_cd_2,pr_interest_keyword_cd_3,ch_interest_keyword_cd_1,ch_interest_keyword_cd_2,ch_interest_keyword_cd_3
7,7,3,20.0,2022-03-01 11:56:53,2022-03-01 12:31:56,2022-03-01 12:06:50,-1 days +23:34:54,-1506.0,False,119.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
11,11,3,21.0,2022-03-01 11:56:53,2022-03-01 12:34:11,2022-03-01 12:08:54,-1 days +23:34:43,-1517.0,False,119.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
38,38,5,74.0,2022-03-06 11:09:42,2022-03-06 11:15:55,2022-03-06 11:15:09,-1 days +23:59:14,-46.0,False,301.0,...,0.0,4.0,M,5,P07,P08,P06,K05,K08,K09
42,42,5,75.0,2022-03-06 11:09:42,2022-03-06 11:20:58,2022-03-06 11:15:32,-1 days +23:54:34,-326.0,False,19.0,...,0.0,1.0,M,5,P07,P08,P06,K05,K08,K09
73,73,5,102.0,2022-03-06 14:23:51,2022-03-06 14:55:39,2022-03-06 14:49:20,-1 days +23:53:41,-379.0,False,80.0,...,0.0,4.0,M,5,P07,P08,P06,K05,K08,K09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2056748,2056748,33032,381.0,2022-04-27 15:50:09,2022-04-27 15:56:42,2022-04-27 15:56:46,0 days 00:00:04,4.0,False,462.0,...,0.0,4.0,F,5,P03,P05,P07,K01,K05,K09
2056749,2056749,33032,381.0,2022-04-27 15:50:09,2022-04-27 15:56:42,2022-04-27 15:56:48,0 days 00:00:06,6.0,False,463.0,...,0.0,4.0,F,5,P03,P05,P07,K01,K05,K09
2056750,2056750,33032,381.0,2022-04-27 15:50:09,2022-04-27 15:56:42,2022-04-27 15:56:53,0 days 00:00:11,11.0,False,463.0,...,0.0,4.0,F,5,P03,P05,P07,K01,K05,K09
2056751,2056751,33032,381.0,2022-04-27 15:50:09,2022-04-27 15:56:42,2022-04-27 15:56:09,-1 days +23:59:27,-33.0,False,462.0,...,0.0,4.0,F,5,P03,P05,P07,K01,K05,K09


In [164]:
dfv3 = dfv3.drop(sample.index)

In [165]:
dfv3

,Unnamed: 0,profile_id,album_id,ss_id,log_time_x,log_time_y,diff,diff_ss,diff_ox,watch_time,...,search_ox,prefer,sex,age,pr_interest_keyword_cd_1,pr_interest_keyword_cd_2,pr_interest_keyword_cd_3,ch_interest_keyword_cd_1,ch_interest_keyword_cd_2,ch_interest_keyword_cd_3
0,0,3,15.0,2022-03-01 11:56:53,2022-03-01 11:57:19,2022-03-01 11:58:05,0 days 00:00:46,46.0,True,46.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
1,1,3,16.0,2022-03-01 11:56:53,2022-03-01 11:58:09,2022-03-01 11:59:54,0 days 00:01:45,105.0,False,104.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
2,2,3,17.0,2022-03-01 11:56:53,2022-03-01 11:59:58,2022-03-01 12:01:14,0 days 00:01:16,76.0,True,76.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
3,3,3,18.0,2022-03-01 11:56:53,2022-03-01 12:01:18,2022-03-01 12:02:26,0 days 00:01:08,68.0,False,67.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
4,4,3,19.0,2022-03-01 11:56:53,2022-03-01 12:02:29,2022-03-01 12:04:00,0 days 00:01:31,91.0,False,90.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2056833,2056833,33032,373.0,2022-04-27 15:50:09,2022-04-27 15:51:05,2022-04-27 15:51:12,0:00:07,7.0,True,7.0,...,0.0,1.0,F,5,P03,P05,P07,K01,K05,K09
2056834,2056834,33032,3128.0,2022-04-27 15:50:09,2022-04-27 15:51:51,2022-04-27 15:51:53,0:00:02,2.0,True,2.0,...,0.0,1.0,F,5,P03,P05,P07,K01,K05,K09
2056835,2056835,33032,393.0,2022-04-27 15:50:09,2022-04-27 15:51:47,2022-04-27 15:53:08,0:01:21,81.0,True,81.0,...,0.0,1.0,F,5,P03,P05,P07,K01,K05,K09
2056836,2056836,33032,3625.0,2022-04-27 15:50:09,2022-04-27 15:54:48,2022-04-27 15:54:57,0:00:09,9.0,True,9.0,...,0.0,1.0,F,5,P03,P05,P07,K01,K05,K09


In [166]:
dfv3.isnull().sum().to_frame()

,0
Unnamed: 0,0
profile_id,0
album_id,26
ss_id,26
log_time_x,26
log_time_y,101969
diff,101969
diff_ss,101969
diff_ox,0
watch_time,101969


In [167]:
dfv3.to_csv("all_df_v3.csv")

In [168]:
#merged1 = pd.read_csv(os.path.join(data_path, 'all_df_v2.csv'), encoding='utf-8')
test = pd.read_csv('all_df_v3.csv')

In [169]:
test

,Unnamed: 0.1,Unnamed: 0,profile_id,album_id,ss_id,log_time_x,log_time_y,diff,diff_ss,diff_ox,...,search_ox,prefer,sex,age,pr_interest_keyword_cd_1,pr_interest_keyword_cd_2,pr_interest_keyword_cd_3,ch_interest_keyword_cd_1,ch_interest_keyword_cd_2,ch_interest_keyword_cd_3
0,0,0,3,15.0,2022-03-01 11:56:53,2022-03-01 11:57:19,2022-03-01 11:58:05,0 days 00:00:46,46.0,True,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
1,1,1,3,16.0,2022-03-01 11:56:53,2022-03-01 11:58:09,2022-03-01 11:59:54,0 days 00:01:45,105.0,False,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
2,2,2,3,17.0,2022-03-01 11:56:53,2022-03-01 11:59:58,2022-03-01 12:01:14,0 days 00:01:16,76.0,True,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
3,3,3,3,18.0,2022-03-01 11:56:53,2022-03-01 12:01:18,2022-03-01 12:02:26,0 days 00:01:08,68.0,False,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
4,4,4,3,19.0,2022-03-01 11:56:53,2022-03-01 12:02:29,2022-03-01 12:04:00,0 days 00:01:31,91.0,False,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1400717,2056833,2056833,33032,373.0,2022-04-27 15:50:09,2022-04-27 15:51:05,2022-04-27 15:51:12,0:00:07,7.0,True,...,0.0,1.0,F,5,P03,P05,P07,K01,K05,K09
1400718,2056834,2056834,33032,3128.0,2022-04-27 15:50:09,2022-04-27 15:51:51,2022-04-27 15:51:53,0:00:02,2.0,True,...,0.0,1.0,F,5,P03,P05,P07,K01,K05,K09
1400719,2056835,2056835,33032,393.0,2022-04-27 15:50:09,2022-04-27 15:51:47,2022-04-27 15:53:08,0:01:21,81.0,True,...,0.0,1.0,F,5,P03,P05,P07,K01,K05,K09
1400720,2056836,2056836,33032,3625.0,2022-04-27 15:50:09,2022-04-27 15:54:48,2022-04-27 15:54:57,0:00:09,9.0,True,...,0.0,1.0,F,5,P03,P05,P07,K01,K05,K09


In [170]:
test.isnull().sum().to_frame()

,0
Unnamed: 0.1,0
Unnamed: 0,0
profile_id,0
album_id,26
ss_id,26
log_time_x,26
log_time_y,101969
diff,101969
diff_ss,101969
diff_ox,0


In [ ]:
import zipfile
import os


## change working directory
base_dir = ''../data'
os.chdir(base_dir)

## check the current working directory
os.getcwd()

[Out] '/Users/ihongdon/Downloads'


## show the lists of files in the current working directory
os.listdir()

['sample_2.txt', 'sample_3.txt', 'sample_1.txt']